# Beschermde gebieden: inzichtelijk maken van berijden door zwaar vrachtverkeer van kwetsbare assets
Op basis van de bereikbaarheidscasus (verkeersborden) kunnen we gebieden aanwijzen waar een gewichtsbeperking (totaal of aslast) geldt. De aanname is dat deze beperking een kwetsbaarheid in het wegdeel impliceert, bijvoorbeeld een kade of brug. Vrachtverkeer dat aan deze beperkingen voeldoet is toegestaan, maar nog steeds niet wenselijk gezien het kan leiden tot extra onderhoud. 

Inputs voor deze casus zijn:
- Alle wegdelen (met ID)
- (ID van) kwetsbare wegdelen
- GPS locaties van waar vrachtverkeer heeft gereden (op basis van Bridgestone data)
## Nog te doen
- Koppeling van traces is nu in absolute counts van distinct sessionIDs, niet genormaliseerd naar weglengte
- Korte wegdelen hebben daardoor een onder-represented intensity, omdat de kans klein is dat de vrachtwagen gelogd is

In [1]:
import psycopg2
try:
    postgres_pw = TokenLibrary.getSecret("redacted", "redacted", "redacted")
    conn = psycopg2.connect(dbname="rdt",user="user",host="host",password=postgres_pw)
    conn.autocommit = True
    cur=conn.cursor()
    print("Database connected")
except:
    print("I am unable to connect to the database")

## Identificeren kwetsbare wegdelen.
Als er een gewichtsbeperking (aslast of lading) geldt, dan impliceert dit mogelijk een kwetbaar wegdeel. Dit kan bijvoorbeeld een brug of kade zijn.

Berekenen kan door het bereikbaarheidsalgoritme te runnen, maar waar voertuigtype en lengte/breedte/hoogte wordt genegeerd

In [8]:
# Changing the Geom of kwetsbare wegdelen
sql='''drop table if exists stg_aandachtsgebieden.ndw_kwetsbarewegdelen;
CREATE TABLE stg_aandachtsgebieden.ndw_kwetsbarewegdelen
AS (
select v.id,
    CASE 
        when v.bereikbaar_status_code = 333 then 333
        when v.bereikbaar_status_code = 222 then 222
        else 999
    end as bereikbaar_status_code,
    -- ST_AsGeoJson(ST_AsText(ST_Transform(v.geom, 4326), 6)) as geom
    v.geom as geom
from (
    select abs(n.id) as id,
        max(case
                when n.cost is NULL then 333  --Eenrichtingsverkeer?
                when routing.agg_cost is null then 222 --Onbereikbaar via andere links?
                -- Wegvak niet begaanbaar voor dit voertuig?
                when 
                    n.c01 is not null -- verboden in te rijden
                    or n.c20 is not null -- gewichtsbeperking
                    or n.c21 is not null -- gewichtsbeperking
                then 222
                else 999
            end
        ) as bereikbaar_status_code,
        g.geom as geom
    from prd_ndw_borden.nwb_wegvakken_bebording AS n
    left join (
        SELECT start_vid as source,
            end_vid as target,
            agg_cost
        FROM pgr_dijkstraCost('
            select id, source, target, cost
            from prd_ndw_borden.nwb_wegvakken_bebording
            where cost > 0
            and (
                -- Alleen links zonder gewichtsbeperking gebruiken
                c20 is null
                and c21 is null
                and ((c01 < 1) or c01 is null)
            )',
            208419012,
            array(
                select node
                from prd_ndw_wegen.nwb_wegvakken_nodes
            )
        )
    ) as routing on n.source = routing.target

    left join prd_ndw_borden.nwb_wegvakken_bebording AS g
        on abs(n.id) = g.id
        where abs(n.id) in (
            select id from prd_ndw_borden.nwb_wegvakken_bebording
            where id > 0
        )
        and n.cost > 0

    group by abs(n.id), g.geom
    order by abs(n.id)
) AS v
where v.bereikbaar_status_code <> 999
);'''
cur.execute(sql)

In [9]:
# Changing the Geom of kwetsbare wegdelen
sql='''DROP TABLE IF EXISTS int_aandachtsgebieden.ndw_kwetsbarewegdelen_28992 CASCADE;
CREATE TABLE int_aandachtsgebieden.ndw_kwetsbarewegdelen_28992
AS (
    select  id, ST_TRANSFORM(geom, 28992) as geom
    from stg_aandachtsgebieden.ndw_kwetsbarewegdelen
);'''
cur.execute(sql)

## Koppelen van Bridgestone vrachtverkeer data aan wegdelen
Tabel ndw_bereikbaarheid.traces_truck_raw bevat alleen data binnen 10 meter rondom een kwetsbaar wegdeel en bevat lon/lat maar geen geometry

In [5]:
# Create bbox for use in traces subselection
sql='''DROP TABLE IF EXISTS int_aandachtsgebieden.ndw_kwetsbarewegdelen_bbox CASCADE;
CREATE TABLE int_aandachtsgebieden.ndw_kwetsbarewegdelen_bbox
AS (
    SELECT id,
        ROUND(SPLIT_PART(coords,' ',1)::float*10^6)::int as ll_lon,
        ROUND(SPLIT_PART(coords,' ',2)::float*10^6)::int as ll_lat,
        ROUND(SPLIT_PART(coords,' ',3)::float*10^6)::int as ur_lon,
        ROUND(SPLIT_PART(coords,' ',4)::float*10^6)::int as ur_lat,
        ST_MakeEnvelope(SPLIT_PART(coords,' ',1)::float,SPLIT_PART(coords,' ',2)::float,SPLIT_PART(coords,' ',3)::float,SPLIT_PART(coords,' ',4)::float,4326) as geom
    FROM (
        select id, REPLACE(SUBSTRING(box, 5, length(box)-5),',',' ') as coords
        FROM (
            SELECT id, ST_Extent(ST_TRANSFORM(ST_Buffer(geom,10),4326))::varchar as box
            from int_aandachtsgebieden.ndw_kwetsbarewegdelen_28992
            GROUP BY id
        ) d
    ) c
    -- Texel uit data halen omdat het een los wegennetwerk is
    WHERE   ROUND(SPLIT_PART(coords,' ',2)::float*10^6)::int < 52980000
    OR     ROUND(SPLIT_PART(coords,' ',1)::float*10^6)::int > 4915000
);'''
cur.execute(sql)

In [11]:
# Uitfilteren van Bridgestone data, van trucks, binnen een bbox
%run RDT_traces_vrachtverkeer_NH

In [ ]:
# Truck traces contain latitude longitude, but not a geometry.
# Geometry can be constructed from latlong
sql='''DROP TABLE IF EXISTS int_aandachtsgebieden.traces_truck_raw_geom CASCADE;
CREATE TABLE int_aandachtsgebieden.traces_truck_raw_geom
AS (
    SELECT traces.*, ST_Transform(ST_SetSRID(ST_MakePoint(round(longitude/1E6, 7)::float, round(latitude/1E6, 7)::float), 4326), 28992) as geom
    FROM stg_aandachtsgebieden.traces_truck_raw as traces
);'''
cur.execute(sql)

Selecteer alleen de wegdelen binnen 10m vanaf een kwetsbaar wegdeel. Dit versnelt de query waarbij traces aan een wegdeel worden gekoppeld.

De buffer van 10m is nodig omdat de traces ook in een buffer rondom de wegdelen zijn genomen.

In [ ]:
# Create table with roads close to Kwetsbare wegdelen
sql='''DROP TABLE IF EXISTS int_aandachtsgebieden.ndw_kwetsbarewegdelen_buffer10m_28992 CASCADE;
CREATE TABLE int_aandachtsgebieden.ndw_kwetsbarewegdelen_buffer10m_28992
AS (
    SELECT id, geom
    FROM (
        SELECT id, ST_Transform(geom,28992) as geom
        FROM prd_ndw_wegen.nwb_wegvakken_directed as r
    ) as roads
    WHERE ST_DWithin(roads.geom, (
        -- Get closest KwetsbaarWegdeel to current road
        SELECT roads_kwetsbaar.geom
        FROM int_aandachtsgebieden.ndw_kwetsbarewegdelen_28992 as roads_kwetsbaar
        ORDER BY roads_kwetsbaar.geom <-> roads.geom
        LIMIT 1)
    ,10) -- 10m
);'''
cur.execute(sql)

Elke trace wordt gekoppeld aan een road_id

In [ ]:
# Find ID of closest road and add it to the trace data
sql='''DROP TABLE IF EXISTS int_aandachtsgebieden.traces_truck_raw_roadid CASCADE;
CREATE TABLE int_aandachtsgebieden.traces_truck_raw_roadid
AS (
    SELECT traces.*, roads_closest.id
    FROM int_aandachtsgebieden.traces_truck_raw_geom as traces
    CROSS JOIN LATERAL (
        SELECT roads_selection.id, roads_selection.geom, roads_selection.geom <-> traces.geom AS dist
        FROM int_aandachtsgebieden.ndw_kwetsbarewegdelen_buffer10m_28992 as roads_selection
        ORDER BY dist
        LIMIT 1
    ) as roads_closest
);'''
cur.execute(sql)

Tel het aantal unieke traces per wegsegment (met een ID dat ook in kwetsbare wegdelen voorkomt) => intensiteit

In [ ]:
sql='''DROP TABLE IF EXISTS int_aandachtsgebieden.traces_truck_merged CASCADE;
CREATE TABLE int_aandachtsgebieden.traces_truck_merged
AS (
    SELECT merge_roads.intensity, merge_roads.id, merge_roads.month, roads_kwetsbaar.geom
    FROM (
        SELECT SUM(raw_merge.intensity) as intensity, id, month
        FROM (
            SELECT count(distinct(session_id)) as intensity, ABS(id) as id, to_char(timestamp, 'YYYY-MM') as month
            FROM int_aandachtsgebieden.traces_truck_raw_roadid
            GROUP BY ABS(id), to_char(timestamp, 'YYYY-MM')
            ) as raw_merge
        GROUP BY raw_merge.id, month
    ) as merge_roads
    INNER JOIN stg_aandachtsgebieden.ndw_kwetsbarewegdelen as roads_kwetsbaar
    ON roads_kwetsbaar.id=merge_roads.id
    ORDER BY merge_roads.intensity DESC
);'''
cur.execute(sql)

In [ ]:
# Data to display as WFS layer
sql='''DROP TABLE IF EXISTS prd_aandachtsgebieden.displayTable_infra;
CREATE TABLE prd_aandachtsgebieden.displayTable_infra AS (
    SELECT  ttm.*, ST_length(ST_TRANSFORM(ttm.geom, 28992)) as weglengte
            ,roads_closest.straatnaam, roads_closest.rvv_modelnummer, roads_closest.tekst_waarde
            ,ws.wvk_id as road_id, ws.wegbehnaam as roadauthority
    FROM int_aandachtsgebieden.traces_truck_merged as ttm
    CROSS JOIN LATERAL (
        SELECT bv3.*, ST_TRANSFORM(bv3.geom, 28992) <-> ST_TRANSFORM(ttm.geom, 28992) AS dist
        FROM prd_ndw_borden.ndw_bebording_verrijkt_2 as bv3
        ORDER BY dist
        LIMIT 1
    ) as roads_closest
    LEFT JOIN stg_ndw_wegen.nwb_raw_subselection as ws
    ON oc.roadid=ws.wvk_id
    ORDER BY ttm.intensity DESC
);'''
cur.execute(sql)